In [25]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import time
import re
import itertools
from tqdm import tqdm

from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.manifold import TSNE
import networkx as nx

import tensorflow as tf
from keras.layers import Dense, Flatten, LSTM, Conv1D, MaxPooling1D, Dropout, Activation
from keras.layers.embeddings import Embedding
import keras 
import scipy
from keras import backend as K
from keras.preprocessing.text import Tokenizer, one_hot
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense,Flatten,Activation,Dropout,Input,Embedding,RepeatVector,Permute,merge,Lambda

from keras.layers import Multiply ,concatenate ,Subtract,add,Add,Reshape                   
from keras.optimizers import RMSprop ,Adam
from keras.preprocessing import sequence
from keras.utils import to_categorical
from keras.callbacks import EarlyStopping
from keras.models import Model

In [2]:
train_data = pd.read_csv('/Users/s0c02nj/Desktop/Personal Edu doc/Competitions /Hike/train/train.csv')
train_feat = pd.read_csv('/Users/s0c02nj/Desktop/Personal Edu doc/Competitions /Hike/train/user_features.csv')

In [3]:
test_data = pd.read_csv('/Users/s0c02nj/Desktop/Personal Edu doc/Competitions /Hike/test.csv')

In [4]:
train_data.shape

(70661802, 3)

Rows to remove

In [5]:
rem_row = np.load('/Users/s0c02nj/Desktop/Personal Edu doc/Competitions /Hike/same_node_train_ids.npy')

In [6]:
len(rem_row)

903488

In [44]:
#train_data_new = train_data[0:706618]
706618*2

1413236

In [80]:
train_data_new = train_data[15132360:35132360]

All nodes are present

In [81]:
train_data1 = train_data_new[['node1_id']]
train_data2 = train_data_new[['node2_id']]

y_train = train_data_new['is_chat']

In [82]:
y_cat = to_categorical(y_train)

In [48]:
test_data1 = test_data[['node1_id']]
test_data2 = test_data[['node2_id']]


Bringing the Train Features

In [83]:
train_feat1 = pd.merge(left = train_data1, right= train_feat, how='left', 
                 left_on=['node1_id'], right_on=['node_id'])

In [84]:
train_feat2 = pd.merge(left = train_data2, right= train_feat, how='left', 
                 left_on=['node2_id'], right_on=['node_id'])

Bringing Test Features

In [51]:
test_feat1 = pd.merge(left = test_data1, right= train_feat, how='left', 
                 left_on=['node1_id'], right_on=['node_id'])

In [52]:
test_feat2 = pd.merge(left = test_data2, right= train_feat, how='left', 
                 left_on=['node2_id'], right_on=['node_id'])

Additional features

In [53]:
#train_feat1[train_feat1['node1_id']== 8446602]

Standardizing Columns

In [70]:
cols_std = ['f1','f2','f3','f4','f5', 'f6','f7', 'f8', 'f9', 'f10', 'f11', 'f12', 'f13']

In [85]:
train_feat1_fin = train_feat1[cols_std]
train_feat2_fin = train_feat2[cols_std]

In [56]:
test_feat1_fin = test_feat2[cols_std]
test_feat2_fin = test_feat2[cols_std]

Importing the Node vectors

In [ ]:
#path = '/Users/s0c02nj/Desktop/Hike/node2vec.txt'

In [ ]:
# count = 0
# vectors_list = []

# for o in tqdm(open(path)):
    
#     values = o.split()
#     values[0] = int(values[0])
#     values[1:] = list(map(float, values[1:]))
    
#     vectors_list.append(values)

In [ ]:
# cols = []
# for i in range(0,16):
#     cols.append('data'+str(i))

In [ ]:
#cols = ['node_id']+cols

In [ ]:
# node_vec = pd.DataFrame(vectors_list)
# node_vec.columns = cols

In [ ]:
#node_vec = node_vec.dropna()

In [ ]:
#node_vec.index = range(0,len(node_vec))

Joining with the node_vec features

In [58]:
train_feat1_fin = pd.merge(left = train_feat1, right= node_vec, how='left', 
                 left_on=['node1_id'], right_on=['node_id'])

In [ ]:
train_feat2_fin = pd.merge(left = train_feat2, right= node_vec, how='left', 
                 left_on=['node2_id'], right_on=['node_id'])

In [ ]:
test_feat1_fin = pd.merge(left = test_feat1, right= node_vec, how='left', 
                 left_on=['node1_id'], right_on=['node_id'])

In [ ]:
test_feat2_fin = pd.merge(left = test_feat2, right= node_vec, how='left', 
                 left_on=['node2_id'], right_on=['node_id'])

In [ ]:
cols = []
for i in range(0,16):
    cols.append('data'+str(i))
cols_total = cols_std + cols

In [ ]:
train_feat1_fin = train_feat1_fin[cols_total]
train_feat2_fin = train_feat2_fin[cols_total]

In [ ]:
test_feat1_fin = test_feat1_fin[cols_total]
test_feat2_fin = test_feat2_fin[cols_total]

Deep Learning Archi

In [86]:
def model_deep():
    
    #Defining the input-----> Continuous Variable
    inputs1 = Input(shape=(13,))
    
    #Defining the input-----> Categorical Variable
    inputs2 = Input(shape=(13,))
    
    #Subtraction
    subtracted = Subtract()([inputs1, inputs2])
    
    #multiply
    multiply =  Multiply()([inputs1, inputs2])
    
    
    #Defining the -----> Complaint-reason
    layer_merge = concatenate([inputs1,inputs2,subtracted,multiply])
    

    #Dense Layer-----> This will be the Weight Matrix W (n*k)
    layer_dense = Dense(150, activation='relu')(layer_merge)
    layer_dense = Dropout(0.5)(layer_dense)
    
    layer_dense = Dense(30, activation='relu')(layer_merge)
    layer_dense = Dropout(0.5)(layer_dense)
    
    layer_dense = Dense(20, activation='relu')(layer_merge)
    layer_dense = Dropout(0.5)(layer_dense)
    
    #Total Loss to Minimize
    loss_total = Dense(2,activation='softmax')(layer_dense)
    
    
    model = Model(inputs=[inputs1,inputs2], outputs= loss_total)
    return model

In [87]:
model = model_deep()
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_9 (InputLayer)            (None, 13)           0                                            
__________________________________________________________________________________________________
input_10 (InputLayer)           (None, 13)           0                                            
__________________________________________________________________________________________________
subtract_4 (Subtract)           (None, 13)           0           input_9[0][0]                    
                                                                 input_10[0][0]                   
__________________________________________________________________________________________________
multiply_4 (Multiply)           (None, 13)           0           input_9[0][0]                    
          

In [88]:
def auc(y_true, y_pred):
    auc = tf.metrics.auc(y_true, y_pred)[1]
    K.get_session().run(tf.local_variables_initializer())
    return auc

In [89]:
model.compile(loss = "binary_crossentropy", optimizer = Adam(lr=0.001), metrics = [auc])

In [90]:
#Deep Learning Model
history = model.fit([train_feat1_fin,
                     train_feat2_fin] ,
                     y_cat, batch_size = 1024, epochs = 4, validation_split=0.1,
                     class_weight = {0:2 ,1:40}, 
                     verbose = 1)


Train on 18000000 samples, validate on 2000000 samples
Epoch 1/4
18000000/18000000 [==============================] - 145s 8us/step - loss: 2.1395 - auc: 0.8964 - val_loss: 1.7214 - val_auc: 0.9081
Epoch 2/4
18000000/18000000 [==============================] - 141s 8us/step - loss: 1.7825 - auc: 0.9044 - val_loss: 1.6973 - val_auc: 0.9031
Epoch 3/4
18000000/18000000 [==============================] - 165s 9us/step - loss: 1.7708 - auc: 0.9027 - val_loss: 1.6823 - val_auc: 0.9020
Epoch 4/4
18000000/18000000 [==============================] - 157s 9us/step - loss: 1.7643 - auc: 0.9017 - val_loss: 1.6766 - val_auc: 0.9013


In [42]:
#Tets
y_prob1 = model.predict([test_feat1_fin,test_feat2_fin])

In [43]:
y_ones1 = y_prob1[:,1]

In [64]:
#Tets
y_prob2 = model.predict([test_feat1_fin,test_feat2_fin])

In [78]:
y_ones2 = y_prob2[:,1]

In [79]:
#Tets
y_prob3 = model.predict([test_feat1_fin,test_feat2_fin])

In [91]:
y_ones3 = y_prob3[:,1]

In [92]:
#Tets
y_prob4 = model.predict([test_feat1_fin,test_feat2_fin])

In [93]:
y_ones4 = y_prob4[:,1]

In [98]:
y_pred = (y_ones1+y_ones2+y_ones3+y_ones4)/4.0

In [101]:
sub_df = test_data.copy()

In [103]:
sub_df['is_chat'] = y_pred

In [104]:
sub_df = sub_df[['id','is_chat']]

In [106]:
#z = np.load('/Users/s0c02nj/Desktop/Hike/same_node_ids.npy')
#z_set = set(z)

In [ ]:
out = []
for i in tqdm(range(0,len(sub_df))):
    
    i_id = sub_df['id'].iloc[i]
    
    if i_id in z_set:
        k=0
    else:
        k = sub_df['is_chat'].iloc[i]
        
    out.append(k)

In [ ]:
sub_df['is_chat'] = out

In [108]:
sub_df.to_csv('/Users/s0c02nj/Desktop/Personal Edu doc/Competitions /Hike/Submission6.csv',index=False)